In [2]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn import (metrics, linear_model, preprocessing)
from sklearn.model_selection import train_test_split 
import xgboost as xgb 
from matplotlib import pyplot
# feature selection    make sure to update xgboost package
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, RepeatedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import uniform,randint
from xgboost import plot_importance

In [3]:
path = './../data/Trainingonline.csv'
df = pd.read_csv(path, na_values='NA')
print (list(df.columns))
df.head(6)

# Las primeras 12 columnas deberían ser nuestra feature.

['Outcome_M1', 'Outcome_M2', 'Outcome_M3', 'Outcome_M4', 'Outcome_M5', 'Outcome_M6', 'Outcome_M7', 'Outcome_M8', 'Outcome_M9', 'Outcome_M10', 'Outcome_M11', 'Outcome_M12', 'Cat_1', 'Date_1', 'Quan_1', 'Quan_2', 'Quan_3', 'Quan_4', 'Date_2', 'Quan_5', 'Quan_6', 'Quan_7', 'Quan_8', 'Quan_9', 'Quan_10', 'Quan_11', 'Quan_12', 'Quan_13', 'Quan_14', 'Cat_2', 'Cat_3', 'Cat_4', 'Cat_5', 'Cat_6', 'Cat_7', 'Cat_8', 'Cat_9', 'Cat_10', 'Cat_11', 'Cat_12', 'Cat_13', 'Cat_15', 'Cat_16', 'Quan_15', 'Cat_17', 'Cat_18', 'Cat_19', 'Cat_20', 'Cat_21', 'Cat_22', 'Cat_23', 'Cat_24', 'Cat_25', 'Cat_26', 'Cat_27', 'Cat_28', 'Cat_29', 'Cat_30', 'Cat_31', 'Cat_32', 'Cat_33', 'Cat_34', 'Cat_35', 'Cat_36', 'Cat_37', 'Cat_38', 'Cat_39', 'Cat_40', 'Cat_41', 'Cat_42', 'Cat_43', 'Cat_44', 'Cat_45', 'Cat_46', 'Cat_47', 'Cat_48', 'Cat_49', 'Cat_50', 'Cat_51', 'Cat_52', 'Cat_53', 'Cat_54', 'Cat_55', 'Cat_56', 'Cat_57', 'Cat_58', 'Cat_59', 'Cat_60', 'Cat_61', 'Cat_62', 'Cat_63', 'Cat_64', 'Cat_65', 'Cat_66', 'Cat_67', '

,Outcome_M1,Outcome_M2,Outcome_M3,Outcome_M4,Outcome_M5,Outcome_M6,Outcome_M7,Outcome_M8,Outcome_M9,Outcome_M10,...,Cat_505,Cat_506,Cat_507,Cat_508,Cat_509,Cat_510,Cat_511,Cat_512,Cat_513,Cat_514
0,10000,3000.0,8000.0,3000.0,3000.0,2000.0,2000.0,500.0,500.0,500.0,...,0,0,0,1,0,1,0,0,0,1
1,8000,8000.0,2000.0,3000.0,2000.0,500.0,2000.0,2000.0,3000.0,500.0,...,0,0,0,1,0,0,0,0,0,1
2,5000,15000.0,8000.0,3000.0,3000.0,2000.0,2000.0,2000.0,2000.0,500.0,...,0,0,0,1,0,0,0,0,0,1
3,35000,8000.0,8000.0,5000.0,5000.0,3000.0,2000.0,3000.0,6000.0,2000.0,...,0,0,0,1,0,1,0,0,0,1
4,16000,2000.0,2000.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,...,0,0,0,1,0,1,0,0,0,1
5,32000,6000.0,2000.0,2000.0,2000.0,500.0,500.0,500.0,2000.0,2000.0,...,0,0,0,1,0,1,0,0,0,1


In [4]:
# we want to check how many missing values for each column
missing = df.isnull().sum()

# from 12th element in array (i.e. 13th column) are model label features
print (missing[12:20])

# give missing # for all features, and those feature names
# varlist is a list containing the names (str) that have missing values
missingnew = missing[12:]
withmissing = missingnew[missing>0]
varlist = list(withmissing.index)

# create missing dummy for each column with missing values
for varname in varlist:
        missdummy = str(varname)+'_missdummy'
        df[missdummy] = pd.isnull(df[str(varname)])+0

# perform missing value imputation   
for varname in varlist:
      df[str(varname)].fillna(df[str(varname)].median(), inplace=True)


Cat_1       0
Date_1      0
Quan_1      0
Quan_2     12
Quan_3      5
Quan_4     79
Date_2      3
Quan_5    474
dtype: int64


In [5]:
# we prove that Date_1>Date_2
print (np.min(df.Date_1-df.Date_2))
print (np.max(df.Date_1-df.Date_2))
print (np.mean(df.Date_1-df.Date_2))

33.0
3240.0
525.3655126498003


In [6]:
# The shape of original data frame is (751, 584) (584 features), 
# but the first 12 columns are are dependents variables (model targets)
# So we should transform the first 12 columns into 12 rows: one row -> 12 rowa
# also we will use month as a predictor.


Xlist=[]
Ylist=[]

In [7]:
# To create the data frame we want, we iterate over rows of the dataframe
# create X matrix target variable, they now become np array
# at the same time, the month of the year is added as a predictor
m =[]
for index, row in enumerate(df.values):
     for j in range(0,12):
            if not np.isnan(row[j]): 
                Ylist.append(row[j])   
                r = list(row[12:572])
                month = [j+1]
                r.extend(month)
                vlist = np.array(r)
                Xlist.append(vlist)  # append a month of year
                
X = np.array(Xlist)
y = np.log(np.array(Ylist)+1)

# show matrix for ind and dep vars
print (X.shape)
print (y.shape)

(8421, 561)
(8421,)


In [8]:
# make binary variables from categorical variable
# these are the colunms after we remove the first 12 target columns

names=list(df.columns.values)[12:558]

# get the list categorical features only containing 
# the 'Cat_', categorical variables
cat = []
for j in range(0,len(names)):
    if "Cat_" in names[j]: 
         cat.extend([j])

# the # of categorical features            
print (len(cat))


513


In [9]:
# make binary variables from categorical variable
# we first use label encoder

from sklearn import (metrics, linear_model, preprocessing)

X_cp = X.copy()

for i in cat:
        encoder = preprocessing.LabelEncoder()
        encoder.fit(X_cp[:,i])
        s = encoder.transform(X_cp[:,i])    
        array_category = np.array(pd.get_dummies(s))
        if i==cat[0]:
             X_cp_cat = array_category.copy()
        else:                  
             X_cp_cat = np.concatenate([X_cp_cat, array_category], axis=1)

# remove cat variables        
for i in cat:
        np.delete(X_cp, np.s_[i], 1)
        
X_new = np.concatenate([X_cp, X_cp_cat], axis=1)
 
print ('X dimension', X_new.shape)
print ('Y dimension', y.shape)


X dimension (8421, 2435)
Y dimension (8421,)


In [10]:
L = X_new.shape[1]
v = ['var_' + str(j) for j in range(L)]
X_new_df = pd.DataFrame(X_new, columns = v )
X_new_df.columns
X_new_df.shape


reg = xgb.XGBRegressor()
reg.fit(X_new_df, y)  

# get feature importance and ranking
important_values = reg.feature_importances_
sorted_idx = np.argsort(important_values)[::-1]

# remove features with importance = 0
important_var_gain = [(X_new_df.columns[index], important_values[index]) for index in sorted_idx if important_values[index] > 0]
# we have 400 features with importance value>0
len(important_var_gain)

# restructure data and refit
important_varlist = [it[0] for it in important_var_gain]
X_new_df = X_new_df[important_varlist]
X_new_df.shape

reg = xgb.XGBRegressor()
reg.fit(X_new_df, y) 

important_values = reg.feature_importances_
sorted_idx = np.argsort(important_values)[::-1]
important_var_gain = [(X_new_df.columns[index], important_values[index]) for index in sorted_idx]



In [11]:
# the threshold used is 1e-5. Otherwise, “mean” is used by default.
modelselect = SelectFromModel(reg, prefit=True)
X_new = modelselect.transform(X_new_df)
X_new.shape

e:\udemy\xgboost\xgboost_env\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


(8421, 134)

In [12]:
# we'd better use data frame
chosen = modelselect.get_support()
chosen_names = [list(X_new_df.columns)[j] for j in range(len(chosen)) if chosen[j] == True]
len(chosen_names)
X = X_new_df[chosen_names]
list(X.columns)
X.shape

(8421, 134)

In [13]:
####################fit xgboost regressor#######################

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.25, random_state = 19) 

xgbreg = xgb.XGBRegressor(
    learning_rate =0.02,
    n_estimators = 2500,
    max_depth = 7,
    objective= 'reg:squarederror',
    min_child_weight=5, 
    gamma=0.05, 
    subsample=0.8, 
    colsample_bytree=0.8,
    seed = 9,
    tree_method='gpu_hist')


In [14]:
# fitting: using early_stopping_rounds we get the best n_estimators
xgbreg.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="rmse", eval_set=[(X_valid, y_valid)])


e:\udemy\xgboost\xgboost_env\Lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
e:\udemy\xgboost\xgboost_env\Lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:6.77779
[1]	validation_0-rmse:6.64417
[2]	validation_0-rmse:6.51330
[3]	validation_0-rmse:6.38620
[4]	validation_0-rmse:6.26078
[5]	validation_0-rmse:6.13911
[6]	validation_0-rmse:6.01865
[7]	validation_0-rmse:5.90083
[8]	validation_0-rmse:5.78504
[9]	validation_0-rmse:5.67176
[10]	validation_0-rmse:5.56055
[11]	validation_0-rmse:5.45160
[12]	validation_0-rmse:5.34567
[13]	validation_0-rmse:5.24124
[14]	validation_0-rmse:5.13863
[15]	validation_0-rmse:5.03836
[16]	validation_0-rmse:4.94076
[17]	validation_0-rmse:4.84467
[18]	validation_0-rmse:4.75020
[19]	validation_0-rmse:4.65784
[20]	validation_0-rmse:4.56729
[21]	validation_0-rmse:4.47834
[22]	validation_0-rmse:4.39134
[23]	validation_0-rmse:4.30663
[24]	validation_0-rmse:4.22325
[25]	validation_0-rmse:4.14143
[26]	validation_0-rmse:4.06149
[27]	validation_0-rmse:3.98396
[28]	validation_0-rmse:3.90694
[29]	validation_0-rmse:3.83161
[30]	validation_0-rmse:3.75774
[31]	validation_0-rmse:3.68557
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.05, gpu_id=0,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.02, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=7, max_leaves=0, min_child_weight=5, missing=nan,
             monotone_constraints='()', n_estimators=2500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=9, ...)

In [15]:
# we get 1450 iteration time
# Other parameter tuning

# RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import uniform,randint

xgbreg = xgb.XGBRegressor(objective = 'reg:squarederror',
                          tree_method='gpu_hist', predictor='gpu_predictor')

param_dist = {'n_estimators': randint(1000, 2000),
              'learning_rate': uniform(0.01, 0.06),
              'subsample': [0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95],
              'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
              'gamma' : [0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9],
              'colsample_bytree': [0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99],
              'min_child_weight': [1, 2, 3, 5, 7]
             }

# verbose: integer
# Controls the verbosity: the higher, the more messages.

regcv = RandomizedSearchCV(xgbreg, param_distributions = param_dist, 
    n_iter = 25, scoring = 'neg_mean_squared_error', cv = 3,
    error_score = 0, verbose = 3, n_jobs = -1)

search = regcv.fit(X_train, y_train)

search.best_params_

Fitting 3 folds for each of 25 candidates, totalling 75 fits


{'colsample_bytree': 0.5,
 'gamma': 0.2,
 'learning_rate': 0.019274663961585843,
 'max_depth': 10,
 'min_child_weight': 2,
 'n_estimators': 1003,
 'subsample': 0.5}

##### BAYESIAN OPT

In [16]:
# Bayesian optimization
# http://krasserm.github.io/2018/03/21/bayesian-optimization/
# https://www.kdnuggets.com/2019/07/xgboost-random-forest-bayesian-optimisation.html
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error


dtrain = xgb.DMatrix(X_train, label=y_train)

def xgb_evaluate(max_depth, gamma, colsample_bytree, eta, n_estimators):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'n_estimators': int(n_estimators),
              'eta': eta,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]


xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 10), 
                                             'gamma': (0, 1),
                                             'eta': (0.01, 0.2),
                                             'n_estimators': (1000, 2000),
                                             'colsample_bytree': (0.4, 0.95)})

In [17]:
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
# X: Points at which EI shall be computed     
xgb_bo.maximize(init_points=5, n_iter=25, acq='ei')  # ei means expected improvement

# Extract the parameters of the best model.


paramslist = pd.DataFrame(xgb_bo.res).sort_values(['target']).params[len(xgb_bo.res) -1]

# then set new parameter


|   iter    |  target   | colsam... |    eta    |   gamma   | max_depth | n_esti... |
-------------------------------------------------------------------------------------
[23:55:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[23:55:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[23:55:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.



C:\Users\zzz\AppData\Local\Temp\ipykernel_29324\3717027863.py:4: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  xgb_bo.maximize(init_points=5, n_iter=25, acq='ei')  # ei means expected improvement


| 1         | -0.5669   | 0.638     | 0.1429    | 0.909     | 7.936     | 1.802e+03 |
[23:55:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[23:55:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[23:55:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

| 2         | -0.5682   | 0.6627    | 0.1042    | 0.2693    | 6.514     | 1.81e+03  |
[23:55:13] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[23:55:13] WARNING: 

NameError: name 'params' is not defined

In [19]:
paramslist

{'colsample_bytree': 0.7552595794229187,
 'eta': 0.1331634606913604,
 'gamma': 0.8393908270668248,
 'max_depth': 3.1856843876563405,
 'n_estimators': 1696.5507331646438}

In [18]:
int(paramslist['max_depth'])

3